# Imports

Libraries

In [28]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch libraries
import torch

#Neural network libraries
import torch.nn as nn
import torch.nn.functional as F

# #Creating optimizer
import torch.optim as optim

# For terminating program (useful for stopping before training is complete and seeing entire result)
import signal
import sys, os

CSV File

In [29]:
script_dir = os.path.abspath('') # absolute file path in Jupyter
file_path = os.path.join(script_dir, "AI EarthHack Dataset.csv") # file path for the text file input

dataset = pd.read_csv(file_path, header=None)


# Zero-Shot Classification

In [30]:
from transformers import pipeline

# TODO look into changing the modle, bart-large-mnli seems to be the most populat for ZSC
# Other options are T5, GPT, and RoBERTa
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Example class descriptions and input text
class_descriptions = ["economy", "solution", "environment"]
input_text = "Find the innovative circular economy business opportunities. The solutions should be relevant and feasible."

# Perform zero-shot classification
result = classifier(input_text, class_descriptions)

# Print the results
print("Input Text:", input_text)
print("Classifications:")
for label, score in zip(result["labels"], result["scores"]):
    print(f"{label}: {score:.3f}")

model.safetensors: 100%|██████████| 1.63G/1.63G [02:57<00:00, 9.16MB/s]
tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 88.1kB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 9.26MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 14.7MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 9.93MB/s]


Input Text: Find the innovative circular economy business opportunities. The solutions should be relevant and feasible.
Classifications:
economy: 0.521
solution: 0.419
environment: 0.060
